## Analyzing Sentiment with Structured Outputs and Inference.net

This notebook demonstrates how to perform sentiment analysis on text data using the Structured Outputs capability of the OpenAI API, specifically leveraging the `meta-llama/llama-3.2-3b-instruct/fp-16` model via the Inference.net platform.

By utilizing JSON Schema within the `response_format` parameter, we ensure that the model's output strictly adheres to a predefined structure, making it reliable for downstream processing.

### Setup

First, let's install the necessary library.

In [ ]:
!pip install openai

We'll import the required libraries and set up the OpenAI client to point to the Inference.net API endpoint using the `baseurl` parameter. Make sure you have your `INFERENCE_API_KEY` set in your environment variables.

In [1]:
import os
import json
from openai import OpenAI

# —— Configuration ——
INFERENCE_API_KEY = os.getenv("INFERENCE_API_KEY")

# Inference.net client
openai = OpenAI(
    base_url="https://api.inference.net/v1",
    api_key=INFERENCE_API_KEY,
)

Let's define a list of example tweets that we will analyze for sentiment.

In [2]:
tweets = [
    # Ellen DeGeneres’s Oscar selfie (March 2, 2014)
    "If only Bradley's arm was longer. Best photo ever. #oscars",
    # Barack Obama’s re‑election celebration (Nov 6, 2012)
    "Four more years. #Obama2012",
    # Donald Trump’s infamous typo (May 31, 2017)
    "Despite the constant negative press covfefe",
    # Elon Musk on taking Tesla private (Aug 7, 2018)
    "Am considering taking Tesla private at $420. Funding secured.",
    # Barack Obama on the Orlando tragedy (June 12, 2016)
    "Shocked and saddened by the news from Orlando. Our hearts go out to all those impacted and we’ll be there to help.",
    # Ariana Grande after Manchester (May 22, 2017)
    "My heart is broken. From the bottom of my heart, I am so sorry. I don’t have the words.",
    # NASA on Curiosity landing (Aug 5, 2012)
    "Curiosity has landed on Mars! 🎉🔴 #MSL"
] 

We define the JSON schema that the sentiment analysis output must follow. This schema requires the output to be a JSON object with two properties: `text` (the original tweet) and `sentiment` (an enum restricted to "positive", "neutral", or "negative"). The `strict: True` parameter ensures strict adherence to this schema.

In [14]:
sentiment_schema = {
    "name": "tweet_sentiment",
    "strict": True,
    "schema": {
        "type": "object",
        "properties": {
            "text":      {"type": "string"},
            "sentiment": {"type": "string", "enum": ["positive", "neutral", "negative"]},
        },
        "required": ["text", "sentiment"],
        "additionalProperties": False
    }
}

This function takes a tweet as input and uses the Chat Completions API with the JSON schema above to analyze the sentiment. The `response_format` parameter, combined with `strict: True` in the schema, guarantees the output will be a valid JSON object conforming to the `sentiment_schema`.

In [15]:
def analyze_sentiment(tweet):
    messages = [
        {"role": "system", "content": "You are a sentiment analysis assistant. Respond in JSON format adhering to the given schema."},
        {"role": "user", "content": tweet}
    ]

    resp = openai.chat.completions.create(
        model="meta-llama/llama-3.2-3b-instruct/fp-16",
        messages=messages,
        response_format={
            "type": "json_schema",
            "json_schema": sentiment_schema
        }
    )
    return json.loads(resp.choices[0].message.content)

Finally, we iterate through our list of tweets and analyze the sentiment of each one using the `analyze_sentiment` function and print the results.

In [16]:
for tweet in tweets:
    print(analyze_sentiment(tweet))

{'text': 'Just spilled coffee on my laptop right before a big meeting. Mondays, am I right? 😩☕️💻', 'sentiment': 'negative'}
{'text': "Can't believe @Raptors pulled off that comeback in OT! What a game 🔥🏀 #WeTheNorth", 'sentiment': 'positive'}
{'text': 'Is it just me, or does no one understand the importance of personal space on the subway?', 'sentiment': 'negative'}
{'text': 'Your kitten is using the litter box consistently, which is a significant accomplishment.', 'sentiment': 'positive'}
{'text': 'Listening to @BillieEilish’s latest album on repeat. Absolute masterpiece. ', 'sentiment': 'positive'}
{'text': 'Waiting 2 hours for my food delivery and they forgot the fries. Literally starving over here. ', 'sentiment': 'negative'}
{'text': 'feeling anxious but motivated to do my part', 'sentiment': 'positive'}


There we go! We just analyzed a bunch of tweets. Scaling this to millions or billions of examples can get challening with a workflow like this. To solve that, check out our webhook and batch apis:
# [Webhooks](https://docs.inference.net/features/asynchronous-inference/webhooks/getting-started-with-webhooks)
# [Batch API](https://docs.inference.net/features/batch-api)